COMMAND ----------

In [1]:
!pip install camelot-py
!pip install opencv-python
!pip install tabula-py==2.9.0
!pip install langchain
!pip install langchain-core==0.1.40
!pip install jpype1

  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.49
    Uninstalling langsmith-0.1.49:
      Successfully uninstalled langsmith-0.1.49
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.1.40 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.92
    Uninstalling langsmith-0.0.92:
      Successfully uninstalled langsmith-0.0.92
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langc

In [22]:
import glob
import json
import os
import re
import time
import warnings
from datetime import datetime
from pathlib import Path
from urllib.parse import urlparse
import yaml

import camelot
import numpy as np
import openai
import pandas as pd
import requests
import tabula
import tiktoken
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import AzureChatOpenAI
from langchain.docstore.document import Document
from langchain.document_loaders import Docx2txtLoader, PyPDFLoader
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers.regex import RegexParser
from langchain.prompts import (ChatPromptTemplate, HumanMessagePromptTemplate,
                               MessagesPlaceholder,
                               SystemMessagePromptTemplate)
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


In [3]:
model = "GPT4"
openai.api_base = os.getenv("OPENAI_BASE_GPT4")
openai.api_key = os.getenv("OPENAI_KEY_GPT4")
openai.api_version = "2024-02-15-preview"
openai_api_key = openai.api_key


llm = AzureChatOpenAI(
    deployment_name="GPT4",
    openai_api_version="2024-02-15-preview",
    openai_api_key= openai.api_key,
    openai_api_base=openai.api_base,
    engine="GPT4",
    temperature = 0
    
)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


### Helper functions

In [4]:
def extract_text_from_file(df, file_path):
    file_name = file_path.split("/")[-1]
    file_type = file_name.split(".")[-1]
    if file_type == "pdf":
        loader = PyPDFLoader(file_path)
    elif file_type == "docx":
        loader = Docx2txtLoader(file_path)
    else:
        return df

    text = ""
    pages = loader.load_and_split()
    for page in pages:
        text += page.page_content

    # Create a new df and concatenate
    new_row = pd.DataFrame({"file": [file_name], "text": [text]})
    df = pd.concat([df, new_row], ignore_index=True)

    return df


def extract_tables(file_path, pages="all", package="tabula"):
    if package == "camelot":
        # Extract tables with camelot
        # flavor could be 'stream' or 'lattice', for documents where tables do not have clear borders, the stream flavor is generally more appropriate.
        tables = camelot.read_pdf(file_path, pages=pages, flavor="stream")
    else:
        tables = tabula.read_pdf(file_path, pages=pages, stream=True, silent=True)

    # Convert tables to JSON
    tables_json = []
    for idx, table in enumerate(tables):

        if package == "camelot":
            page_number = table.parsing_report["page"]
            data = table.df.to_json(orient="records")
        else:
            page_number = ""
            data = table.to_json(orient="records")

        data = {
            "table_number": idx,
            "page_number": page_number,
            "data": data,
        }
        tables_json.append(data)
    return tables_json

COMMAND ----------

In [19]:
folder_path = '../data/raw'
pathlist = Path(folder_path).glob('*.pdf')
filenames = []
for file_path in pathlist:
    filename = os.path.basename(file_path)
    filenames.append(filename)
len(filenames)

1

In [13]:
df = pd.DataFrame()
for filename in filenames:
    file_path = folder_path + "/" + filename
    file_name = os.path.basename(file_path)
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} process {file_name}")
    # Initialize an empty df
    df_file = pd.DataFrame(columns=["file", "text"])
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} extract text")
    try:
        df_file = extract_text_from_file(df_file, file_path)
    except Exception as e:
        print("----Error: cannot extract text")
        print(f"----error: {e}")
    df = pd.concat([df, df_file])
    
df

Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)


2024-04-28 00:47:15 process GPTsareGPTs.pdf
2024-04-28 00:47:15 extract text


,file,text
0,GPTsareGPTs.pdf,WORKING PAPER\nGPTs are GPTs: An Early Look at...


In [18]:

def extract_text_from_file(df, file_path):
    file_name = file_path.split("/")[-1]
    file_type = file_name.split(".")[-1]
    if file_type == "pdf":
        loader = PyPDFLoader(file_path)
    elif file_type == "docx":
        loader = Docx2txtLoader(file_path)
    else:
        return df

    pages = loader.load_and_split()
    for page_number, page in enumerate(pages, start=1):
        # Each page's text is added as a new row in the DataFrame
        new_row = pd.DataFrame({
            "file": [file_name],
            "page_number": [page_number],
            "text": [page.page_content]
        })
        df = pd.concat([df, new_row], ignore_index=True)
    
    return df



df = pd.DataFrame()
for filename in filenames:
    file_path = folder_path + "/" + filename
    file_name = os.path.basename(file_path)
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} process {file_name}")
    # Initialize an empty df
    df_file = pd.DataFrame(columns=["file", "text"])
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} extract text")
    try:
        df_file = extract_text_from_file(df_file, file_path)
    except Exception as e:
        print("----Error: cannot extract text")
        print(f"----error: {e}")
    df = pd.concat([df, df_file])
    
df.head()

Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)


2024-04-28 00:55:29 process GPTsareGPTs.pdf
2024-04-28 00:55:29 extract text


,file,text,page_number
0,GPTsareGPTs.pdf,WORKING PAPER\nGPTs are GPTs: An Early Look at...,1.0
1,GPTsareGPTs.pdf,WORKING PAPER\nFigure 1: Taken directly from G...,2.0
2,GPTsareGPTs.pdf,WORKING PAPER\nlevels in GPT-4 responses and b...,3.0
3,GPTsareGPTs.pdf,technology. Our evidence supports a wider impa...,4.0
4,GPTsareGPTs.pdf,WORKING PAPER\npolicymakers to predict and reg...,5.0


In [128]:
file_path

'../data/raw/GPTsareGPTs.pdf'

In [21]:
for filename in filenames:
    file_path = folder_path + "/" + filename
    file_name = os.path.basename(file_path)
    df_file = pd.DataFrame()
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} process {file_name}")

    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} extract table")

    all_tables = []
    for package in ["camelot", "tabula"]:
        print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} extract table with {package}")
        try:
            tables_from_package = extract_tables(file_path, pages="all", package=package) # list of json
            for table in tables_from_package:
                all_tables.append({"table": table, "source": package})
        except Exception as e:
            print("----Error: cannot extract table")
            print(f"----error: {e}")

# Now you can access each table along with its source
for entry in all_tables:
    print(f"Source: {entry['source']}, Table: {entry['table']}")

Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)


Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)


2024-04-28 10:24:03 process GPTsareGPTs.pdf
2024-04-28 10:24:03 extract table
2024-04-28 10:24:03 extract table with camelot


Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wron

2024-04-28 10:24:07 extract table with tabula
Source: camelot, Table: {'table_number': 0, 'page_number': 1, 'data': '[{"0":"Abstract","1":""},{"0":"","1":"We investigate the potential implications of large language models (LLMs), such as Generative Pre-"},{"0":"trained Transformers (GPTs), on the U.S. labor market, focusing on the increased capabilities arising from","1":""},{"0":"LLM-powered software compared to LLMs on their own. Using a new rubric, we assess occupations based","1":""},{"0":"","1":"on their alignment with LLM capabilities, integrating both human expertise and GPT-4 classi\\ufb01cations."},{"0":"Our \\ufb01ndings reveal that around 80% of the U.S. workforce could have at least 10% of their work tasks","1":""},{"0":"a\\ufb00ected by the introduction of LLMs, while approximately 19% of workers may see at least 50% of their","1":""},{"0":"","1":"tasks impacted. We do not make predictions about the development or adoption timeline of such LLMs."},{"0":"The projected e\\uf

In [68]:
list(filter(lambda element: element['source']=='tabula',all_tables))[1]['table']

{'table_number': 1,
 'page_number': '',
 'data': '[{"Comparison":"GPT-4, Rubric 1; Human","W":"U","Weighting":"E1","Agreement":"80.8%","Pearson\\u2019s":0.223},{"Comparison":null,"W":"V","Weighting":"E1 + .5*E2","Agreement":"65.6%","Pearson\\u2019s":0.591},{"Comparison":null,"W":"Z","Weighting":"E1 + E2","Agreement":"82.1%","Pearson\\u2019s":0.654},{"Comparison":"GPT-4, Rubric 2; Human","W":"U","Weighting":"E1","Agreement":"81.8%","Pearson\\u2019s":0.221},{"Comparison":null,"W":"V","Weighting":"E1 + .5*E2","Agreement":"65.6%","Pearson\\u2019s":0.538},{"Comparison":null,"W":"Z","Weighting":"E1 + E2","Agreement":"79.5%","Pearson\\u2019s":0.589},{"Comparison":"GPT-4, Rubric 1; GPT-4, Rubric 2","W":"U","Weighting":"E1","Agreement":"91.1%","Pearson\\u2019s":0.611},{"Comparison":null,"W":"V","Weighting":"E1 + .5*E2","Agreement":"76.0%","Pearson\\u2019s":0.705},{"Comparison":null,"W":"Z","Weighting":"E1 + E2","Agreement":"82.4%","Pearson\\u2019s":0.68}]'}

In [65]:
list(filter(lambda element: element['source']=='tabula',all_tables))[1]['table']['data']

'[{"Comparison":"GPT-4, Rubric 1; Human","W":"U","Weighting":"E1","Agreement":"80.8%","Pearson\\u2019s":0.223},{"Comparison":null,"W":"V","Weighting":"E1 + .5*E2","Agreement":"65.6%","Pearson\\u2019s":0.591},{"Comparison":null,"W":"Z","Weighting":"E1 + E2","Agreement":"82.1%","Pearson\\u2019s":0.654},{"Comparison":"GPT-4, Rubric 2; Human","W":"U","Weighting":"E1","Agreement":"81.8%","Pearson\\u2019s":0.221},{"Comparison":null,"W":"V","Weighting":"E1 + .5*E2","Agreement":"65.6%","Pearson\\u2019s":0.538},{"Comparison":null,"W":"Z","Weighting":"E1 + E2","Agreement":"79.5%","Pearson\\u2019s":0.589},{"Comparison":"GPT-4, Rubric 1; GPT-4, Rubric 2","W":"U","Weighting":"E1","Agreement":"91.1%","Pearson\\u2019s":0.611},{"Comparison":null,"W":"V","Weighting":"E1 + .5*E2","Agreement":"76.0%","Pearson\\u2019s":0.705},{"Comparison":null,"W":"Z","Weighting":"E1 + E2","Agreement":"82.4%","Pearson\\u2019s":0.68}]'

In [69]:
list(filter(lambda element: element['source']=='camelot',all_tables))[10]['table']

{'table_number': 10,
 'page_number': 8,
 'data': '[{"0":"corresponded closely to whether an LLM or LLM-powered software could perform the core part of a task or","1":"","2":"","3":"","4":""},{"0":"nearly the entire task.","1":"","2":"","3":"","4":""},{"0":"Comparison","1":"W","2":"Weighting","3":"Agreement","4":"Pearson\\u2019s"},{"0":"GPT-4, Rubric 1; Human","1":"U","2":"E1","3":"80.8%","4":"0.223"},{"0":"","1":"V","2":"E1 + .5*E2","3":"65.6%","4":"0.591"},{"0":"","1":"Z","2":"E1 + E2","3":"82.1%","4":"0.654"},{"0":"GPT-4, Rubric 2; Human","1":"U","2":"E1","3":"81.8%","4":"0.221"},{"0":"","1":"V","2":"E1 + .5*E2","3":"65.6%","4":"0.538"},{"0":"","1":"Z","2":"E1 + E2","3":"79.5%","4":"0.589"},{"0":"GPT-4, Rubric 1; GPT-4, Rubric 2","1":"U","2":"E1","3":"91.1%","4":"0.611"},{"0":"","1":"V","2":"E1 + .5*E2","3":"76.0%","4":"0.705"},{"0":"","1":"Z","2":"E1 + E2","3":"82.4%","4":"0.680"}]'}

In [67]:
list(filter(lambda element: element['source']=='camelot',all_tables))[10]['table']['data']

'[{"0":"corresponded closely to whether an LLM or LLM-powered software could perform the core part of a task or","1":"","2":"","3":"","4":""},{"0":"nearly the entire task.","1":"","2":"","3":"","4":""},{"0":"Comparison","1":"W","2":"Weighting","3":"Agreement","4":"Pearson\\u2019s"},{"0":"GPT-4, Rubric 1; Human","1":"U","2":"E1","3":"80.8%","4":"0.223"},{"0":"","1":"V","2":"E1 + .5*E2","3":"65.6%","4":"0.591"},{"0":"","1":"Z","2":"E1 + E2","3":"82.1%","4":"0.654"},{"0":"GPT-4, Rubric 2; Human","1":"U","2":"E1","3":"81.8%","4":"0.221"},{"0":"","1":"V","2":"E1 + .5*E2","3":"65.6%","4":"0.538"},{"0":"","1":"Z","2":"E1 + E2","3":"79.5%","4":"0.589"},{"0":"GPT-4, Rubric 1; GPT-4, Rubric 2","1":"U","2":"E1","3":"91.1%","4":"0.611"},{"0":"","1":"V","2":"E1 + .5*E2","3":"76.0%","4":"0.705"},{"0":"","1":"Z","2":"E1 + E2","3":"82.4%","4":"0.680"}]'

In [72]:
list(filter(lambda element: element['source']=='camelot',all_tables))[22]

{'table': {'table_number': 22,
  'page_number': 17,
  'data': '[{"0":"Basic Skill","1":"U (\\nU","2":"V","3":"","4":"Z (\\nZ"},{"0":"","1":"std err)","2":"","3":"(std err)","4":"std err)"},{"0":"","1":"","2":"All skill importance scores are normalized to be between 0 and 1.","3":"","4":""},{"0":"Constant","1":"0.082***","2":"","3":"-0.112***","4":"0.300***"},{"0":"","1":"(0.011)","2":"","3":"(0.011)","4":"(0.057)"},{"0":"Active Listening","1":"0.128**","2":"","3":"0.214***","4":"0.449***"},{"0":"","1":"(0.047)","2":"","3":"(0.043)","4":"(0.027)"},{"0":"Mathematics","1":"-0.127***","2":"","3":"0.161***","4":"0.787***"},{"0":"","1":"(0.026)","2":"","3":"(0.021)","4":"(0.049)"},{"0":"Reading Comprehension","1":"0.153***","2":"","3":"0.470***","4":"-0.346***"},{"0":"","1":"(0.041)","2":"","3":"(0.037)","4":"(0.017)"},{"0":"Science","1":"-0.114***","2":"","3":"-0.230***","4":"-0.346***"},{"0":"","1":"(0.014)","2":"","3":"(0.012)","4":"(0.017)"},{"0":"Speaking","1":"-0.028","2":"","3":"0.133

In [103]:
test_case = list(filter(lambda element: element['source']=='camelot',all_tables))[22]['table']['data']
json_data = json.loads(test_case)
df = pd.DataFrame(json_data)
df

,0,1,2,3,4
0,Basic Skill,U (\nU,V,,Z (\nZ
1,,std err),,(std err),std err)
2,,,All skill importance scores are normalized to ...,,
3,Constant,0.082***,,-0.112***,0.300***
4,,(0.011),,(0.011),(0.057)
5,Active Listening,0.128**,,0.214***,0.449***
6,,(0.047),,(0.043),(0.027)
7,Mathematics,-0.127***,,0.161***,0.787***
8,,(0.026),,(0.021),(0.049)
9,Reading Comprehension,0.153***,,0.470***,-0.346***


In [126]:
test_case = list(filter(lambda element: element['source']=='camelot',all_tables))[23]['table']['data']
json_data = json.loads(test_case)
df = pd.DataFrame(json_data, columns=None)
df.drop(df.index[0])

,0,1,2,3,4,5,6,7,8,9,10,11
1,Job,Preparation,Education,Example Occupations,Median,Tot Emp,HUUU,MUUU,HVVV,MVVV,HZZZ,MZZZ
2,Zone,Required,Required,,Income,(000s),,,,,,
3,1,None or little,High school,"Food preparation workers,","$30,230","13,100",0.03,0.04,0.06,0.06,0.09,0.08
4,,(0-3 months),diploma or GED,"dishwashers, ﬂoor sanders",,,,,,,,
5,,,(otional),,,,,,,,,
6,2,Some (3-12,High school,"Orderlies, customer","$38,215","73,962",0.07,0.12,0.16,0.20,0.24,0.27
7,,months),diploma,"service representatives,",,,,,,,,
8,,,,tellers,,,,,,,,
9,3,Medium (1-2,"Vocational school,","Electricians, barbers,","$54,815","37,881",0.11,0.14,0.26,0.32,0.41,0.51
10,,years),"on-the-job training,",medical assistants,,,,,,,,


In [110]:
list(filter(lambda element: element['source']=='camelot',all_tables))[22]['table']['data']

'[{"0":"Basic Skill","1":"U (\\nU","2":"V","3":"","4":"Z (\\nZ"},{"0":"","1":"std err)","2":"","3":"(std err)","4":"std err)"},{"0":"","1":"","2":"All skill importance scores are normalized to be between 0 and 1.","3":"","4":""},{"0":"Constant","1":"0.082***","2":"","3":"-0.112***","4":"0.300***"},{"0":"","1":"(0.011)","2":"","3":"(0.011)","4":"(0.057)"},{"0":"Active Listening","1":"0.128**","2":"","3":"0.214***","4":"0.449***"},{"0":"","1":"(0.047)","2":"","3":"(0.043)","4":"(0.027)"},{"0":"Mathematics","1":"-0.127***","2":"","3":"0.161***","4":"0.787***"},{"0":"","1":"(0.026)","2":"","3":"(0.021)","4":"(0.049)"},{"0":"Reading Comprehension","1":"0.153***","2":"","3":"0.470***","4":"-0.346***"},{"0":"","1":"(0.041)","2":"","3":"(0.037)","4":"(0.017)"},{"0":"Science","1":"-0.114***","2":"","3":"-0.230***","4":"-0.346***"},{"0":"","1":"(0.014)","2":"","3":"(0.012)","4":"(0.017)"},{"0":"Speaking","1":"-0.028","2":"","3":"0.133***","4":"0.294***"},{"0":"","1":"(0.039)","2":"","3":"(0.033)